In [2]:

import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction
import torch
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
from gluonts.transform import AddTimeFeatures
from gluonts.time_feature import time_features_from_frequency_str
from gluonts.time_feature import get_lags_for_frequency
from torch.optim import AdamW


In [3]:
data=pd.read_csv('./data/data_2.csv')

In [10]:
name=['DJUBSWH_x', 'GMECN09', 'DJUBCST', 'SWIX23H',
       'GMESC15', 'NPN416E', 'NPN206E', 'CDOM2UM', 'WTLM3UM', 'NPFL09E',
       'SGENCHT', 'EEXHR19', 'SPGPGHE', 'SGGYEXR', 'DCIENTR', 'GMESA10',
       'NPS404E', 'OIL45SC', 'RJEFNET', 'LCP15MT', 'NPDW17E', 'MLCXDJX',
       'GMESI23', 'NPN104E', 'DJUBSWH_y']
name=name[:20]

In [4]:
def mape(A,F):
  sum=0
  
  for i in range(len(A)):
   

    sum+=abs(A[i] - F[i]) / abs(A[i])
  return 100/len(A)*sum
def smape(A, F):
  sum=0
  for i in range(len(A)):
    sum+=2 * np.abs(F[i] - A[i]) / (np.abs(A[i]) + np.abs(F[i]))

  return 100/len(A) * sum

In [27]:
data.shape

(324, 202)

In [17]:
def transformer(data,period,prediction_period,inputs,batchs,context_length):
    dates = pd.date_range(data.iloc[0,0], periods=period, freq='D')


    ind=pd.PeriodIndex(data=dates,freq='D')
    time_feature=ind.dayofyear.astype(float).values - 1

    past=torch.tensor([[time_feature[:period-prediction_period]]]*inputs)
    past=past.transpose(1,2)
    future=torch.tensor([[time_feature[period-prediction_period:period]]]*inputs)
    future=future.transpose(1,2)

    static=(torch.zeros((inputs,1))).to(torch.float32)
    no=torch.arange(1,inputs+1,1)
    no=no.unsqueeze(1)
    no=no.to(torch.long)
    maskp=torch.ones((inputs,period))
    # maskf=torch.ones((inputs,prediction_period))
    # for i in range(inputs):
    #     for j in range(period):

    #         if data.iloc[j,i]=="":
    #             maskp[i][j]=0
    maskf=maskp[:,period-prediction_period:]
    maskp=maskp[:,:period-prediction_period]
    
   
    whole=torch.tensor(data[name].values)
    input=whole[:period-prediction_period].transpose(0,1)
    target=whole[period-prediction_period:].transpose(0,1)
    traindict={'target':target,'input':input,'past':past,'future':future,'maskp':maskp,'maskf':maskf,'sta':no,'real':static}
    train=Dataset.from_dict(traindict)
    train=train.with_format('torch')

    train_loader = DataLoader(train, batch_size=batchs, shuffle=False)
    lags_sequence = get_lags_for_frequency('1D')
    lags_sequence[15]=period-context_length-prediction_period


    prediction_length = prediction_period
    
    input_size = 1
    config = TimeSeriesTransformerConfig(
        prediction_length=prediction_length,
        context_length=context_length,
        input_size=input_size,
        encoder_layers=4,
        decoder_layers=4,
        d_model=22,
        num_static_categorical_features= 1,
        num_static_real_features= 1,
        num_time_features=1,
        cardinality=[inputs+1],
        lags_sequence=lags_sequence[:16]
    
        
        
    )
    model = TimeSeriesTransformerForPrediction(config)
    
    learning_rate = 0.001  
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    

    optimizer = AdamW(model.parameters(), lr=6e-4, betas=(0.9, 0.95), weight_decay=1e-1)

    model.train()
    for epoch in range(101):
        for ind,batch in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(
                static_categorical_features=batch["sta"]
                if config.num_static_categorical_features > 0
                else None,
                static_real_features=batch["real"]
                if config.num_static_real_features > 0
                else None,
                past_time_features=batch["past"],
                past_values=batch["input"],
                future_time_features=batch["future"],
                future_values=batch["target"],
                past_observed_mask=batch["maskp"],
                future_observed_mask=batch["maskf"],
            )
            loss = outputs.loss

            # Backpropagation
            loss.backward()
            optimizer.step()
        
        if epoch % 50 == 0:
            print(loss.item())

        model.eval()

    forecasts = []

    for batch in train_loader:
        outputs = model.generate(
            static_categorical_features=batch["sta"]
            if config.num_static_categorical_features > 0
            else None,
            static_real_features=batch["real"]
            if config.num_static_real_features > 0
            else None,
            past_time_features=batch["past"],
            past_values=batch["input"],
            future_time_features=batch["future"],
            past_observed_mask=batch["maskp"],
        )
        forecasts.append(outputs.sequences.cpu().numpy())
    forecasts = np.vstack(forecasts)
    foremean=np.mean(forecasts,1)
    test=train['target']

    mapel=[]
    smapel=[]
    for i in range(inputs):
        m=mape(foremean[i],test[i])
        mapel.append(float(m))
        sm=smape(foremean[i],test[i])
        smapel.append(float(sm))
    return foremean,test,mapel,smapel
# get=transformer(data,data.shape[0],7,data.shape[1],25,20)

In [8]:
name=data.columns[1:]

In [24]:
get=transformer(data,data.shape[0],3,200,30,6)

# result= pd.DataFrame({'trans_mape':mapel,'trans_smape':smapel})
# result.to_csv(file)

5.93638801574707
4.075043678283691
3.9512970447540283


'2011/8/22'

In [23]:
res= pd.DataFrame({'trans_mape':get[2],'trans_smape':get[3]})
res.to_csv('D:/timeseries/result/temp2.csv')

# result= pd.DataFrame({'name':name,'day1_pred':get[0][:,0], 'day1_actu':get[1][:,0]
#                        })
# result.to_csv('D:/timeseries/result/temp.csv')

In [39]:
get[0].shape

(200, 7)

In [6]:
from neuralprophet import NeuralProphet


d:\Users\lib\site-packages\plotly_resampler\aggregation\aggregators.py:25: UserWarning: Could not import lttbc; will use a (slower) python alternative.
  warnings.warn("Could not import lttbc; will use a (slower) python alternative.")


In [ ]:
from statsmodels.base.data import handle_missing

def ne_prophet(train_data,test_data,length,fre):
  model = NeuralProphet(
    n_lags=5,
    n_changepoints=20,
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode="multiplicative",
    drop_missing=True)
  metrics = model.fit(train_data, freq=fre) 
  future = model.make_future_dataframe(train_data, periods=len(test_data), n_historic_predictions=len(train_data)) 
  forecast = model.predict(future)

  model.plot(forecast)
  forecast=forecast[length:]
  
  sma=smape(test_data,list(forecast["yhat1"]))
  ma=mape(test_data,list(forecast["yhat1"]))
  return forecast,sma,ma



def run(datas,j):
  # mape_list=[]
  smape_list=[]
  smape_list_np=[]
  for data in datas:

  
    test_data_time=data["Date_"][-j:]
    train_data=data[:-j]
    test_data=list(data["Close_"][-j:])
   
    train_data.columns=["ds","y"]
    
   
    try:
      
    
      
      resultn,sman,smap=ne_prophet(train_data,test_data,len(data)-j,"D")
    
      smape_list.append(smap)
      smape_list_np.append(sman)
    except:
      continue
  return smape_list,smape_list_np



smape_list=[]
smape_list_np=[]
result=[]
test=[]
for i in range(0,200):
  use=data[['ds',name[i]]]
  use=use.rename(columns={name[i]:'y'})

 
  train_data=use[:-7]
  test_data=list(use["y"][-7:])

   
 
  resultn,sman,smap=ne_prophet(train_data,test_data,len(data)-7,"D")
  result.append(resultn)
  test.append(test_data)
  
  smape_list.append(smap)
  smape_list_np.append(sman)





In [ ]:
smape_list

In [ ]:
res= pd.DataFrame({'name':name,'ne_mape':smape_list,'ne_smape':smape_list_np})
res.to_csv('D:/timeseries/result/temp.csv')
res2= pd.DataFrame({'name':name,'day1/7_pred':result[:][0], 'day1/7_actu':test[:,0], 'day2/7_pred':result[:,1], 'day2/7_actu':test[:,1],
                       'day3/7_pred':result[:][2], 'day3/7_actu':test[:,2], 'day4/7_pred':result[:,3], 'day4/7_actu':test[:,3], 
                       'day5/7_pred':result[:,4], 'day5/7_actu':test[:,4], 'day6/7_pred':result[:,5], 'day6/7_actu':test[:,5],
                       'day7/7_pred':result[:,6], 'day7/7_actu':test[:,6]
                       })
res2.to_csv('D:/timeseries/result/pred_2_np.csv')

In [209]:
name=data.columns[1:-5]
dat=data[name]


LSTM

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [6]:
def newrun(df,j,name,ep):





  X = df[name][:-1].values
  Y = df[name][1:].values


  X_norm = X / X[0] 
  Y_norm = Y / Y[0]


  X_train, Y_train = X_norm[:-j], Y_norm[:-j]
  X_test, Y_test = X_norm[-j:], Y_norm[-j:]
  X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
  X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
  


  model = Sequential()
 
  model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
  model.add(Dense(X_train.shape[2]))  
  model.compile(loss='mae', optimizer='adam')
  # print(X_train,"tra")  

 
  history = model.fit(X_train, Y_train, epochs=ep, batch_size=32, validation_data=(X_test, Y_test), verbose=2, shuffle=False)
  
  # for i in range(1,len(name)):
  X_pred=np.array(df.loc[[len(df)-1],name])

    # X_pred=np.concatenate(X_pred,df[name[i]].values[-1])

 
  X_pred_norm = X_pred / X[0]
  Y_pred_norm = model.predict(np.array([X_pred_norm]))
  Y_pred = Y_pred_norm * X[0]




  train_predict = model.predict(X_train)
  test_predict = model.predict(X_test)

 
  smpl=[]
  mpl=[]
  pred=[]
  test=[]

  for i in range(200):
    # plt.figure(figsize=(10,6))
  
    # plt.plot(df['ds'][-j:], Y_test[:,i]*X[0][i], label='Test Actual')
    # plt.plot(df['ds'][-j:], test_predict[:,i]*X[0][i], label='Test Predicted')
    mp=lstmmape(Y_test[:,i]*X[0][i],test_predict[:,i]*X[0][i])
    smp=lstmsmape(Y_test[:,i]*X[0][i],test_predict[:,i]*X[0][i])
    pred.append(test_predict[:,i]*X[0][i])
    test.append(Y_test[:,i]*X[0][i])
   
    # plt.xlabel('Date')
    # plt.ylabel('Price of %s'%name[i])
    # plt.title('Price of %s Prediction'%name[i])
    # plt.legend()
    # plt.show()
    mpl.append(mp)
    smpl.append(smp)
    # print(mpl,smpl,'adas')
    # mpl.append(mp)



  return smpl,mpl,pred,test

def lstmsmape(A, F):
  
  A=list(A)
  F=list(F)
  # print(A,"A","f",F)
  # A=A[:-1]
  # F=F[:-1]
 
  
  sum=0
  for i in range(len(A)):
    sum+=2 * np.abs(F[i] - A[i]) / (np.abs(A[i]) + np.abs(F[i]))
    # print(sum,"as")

  return 100/len(A) * sum

def lstmmape(A,F):
  sum=0
  A=list(A)
  F=list(F)
  # print(A,"A","f",F)
  # A=A[:-1]
  # F=F[:-1]
  
  for i in range(len(A)):
   

    sum+=abs(A[i] - F[i]) / abs(A[i])
  return 100/len(A)*sum

In [211]:
name=['ds', 'DJUBSWH_x', 'GMECN09', 'DJUBCST', 'SWIX23H', 'GMESC15',
       'NPN416E', 'NPN206E', 'CDOM2UM', 'WTLM3UM', 'NPFL09E', 'SGENCHT',
       'EEXHR19', 'SPGPGHE', 'SGGYEXR', 'DCIENTR', 'GMESA10', 'NPS404E',
       'OIL45SC', 'RJEFNET', 'LCP15MT']

In [19]:

smapel=[]
mapel=[]
 
datamain=data
# for j in range(1):
#     epochmp=[]
#     epochsmp=[]
#     for q in range(4):
  
      # nam=['ds']
      # for t in range(5):
         
      
      #   nam.append(name[1+q*5+t])
      # print(nam)
    
      # datamain=dat[nam]
      
  
        
smp,mp,pred,test=newrun(datamain,1,name,40)
      # print(smp,mp)
      # for k in smp:
      #   epochsmp.append(k)
      # for k in mp:
      #   epochmp.append(k)
      # print(epochmp,epochsmp,"dd")
    # break
        

    # mapel.append(epochmp)
    # smapel.append(epochsmp)

Epoch 1/40
11/11 - 3s - loss: 0.9375 - val_loss: 0.6372 - 3s/epoch - 252ms/step
Epoch 2/40
11/11 - 0s - loss: 0.6701 - val_loss: 0.3762 - 57ms/epoch - 5ms/step
Epoch 3/40
11/11 - 0s - loss: 0.3864 - val_loss: 0.2561 - 63ms/epoch - 6ms/step
Epoch 4/40
11/11 - 0s - loss: 0.2173 - val_loss: 0.2219 - 67ms/epoch - 6ms/step
Epoch 5/40
11/11 - 0s - loss: 0.1612 - val_loss: 0.2005 - 66ms/epoch - 6ms/step
Epoch 6/40
11/11 - 0s - loss: 0.1437 - val_loss: 0.1927 - 61ms/epoch - 6ms/step
Epoch 7/40
11/11 - 0s - loss: 0.1365 - val_loss: 0.1905 - 63ms/epoch - 6ms/step
Epoch 8/40
11/11 - 0s - loss: 0.1329 - val_loss: 0.1812 - 69ms/epoch - 6ms/step
Epoch 9/40
11/11 - 0s - loss: 0.1310 - val_loss: 0.1801 - 66ms/epoch - 6ms/step
Epoch 10/40
11/11 - 0s - loss: 0.1317 - val_loss: 0.1743 - 68ms/epoch - 6ms/step
Epoch 11/40
11/11 - 0s - loss: 0.1294 - val_loss: 0.1732 - 63ms/epoch - 6ms/step
Epoch 12/40
11/11 - 0s - loss: 0.1289 - val_loss: 0.1714 - 64ms/epoch - 6ms/step
Epoch 13/40
11/11 - 0s - loss: 0.1290

In [16]:
totalpred=[]
for i in range(1):
    newpred=[j[i] for j in pred]
    totalpred.append(newpred)

totaltest=[]
for i in range(1):
    newtest=[j[i] for j in test]
    totaltest.append(newtest)

In [21]:
len(totalpred)

7

In [18]:
result= pd.DataFrame({'lstm_mape':mp,'lstm_smape':smp})
result.to_csv('D:/timeseries/result/temp_2.csv')
res2= pd.DataFrame({'name':name,'day1/1_pred':totalpred[0], 'day1/1_actu':totaltest[0]})
res2.to_csv('D:/timeseries/result/temp.csv')